<a href="https://colab.research.google.com/github/KNL1979/DS807/blob/main/Part2_RNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading libraries

In [ ]:
!pip install wandb -Uq

In [ ]:
import tensorflow as tf

import wandb
from wandb.keras import WandbCallback

import pandas as pd
#import requests
#import random
import gzip
import json
import io
import os
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn.model_selection import train_test_split

In [ ]:
import psutil

In [ ]:
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
psutil.virtual_memory()[2]

RAM Used (GB): 1.108885504


2.2

# Setting up with Google Drive

In [ ]:
# Mount your google drive where you've saved your assignment folder
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Replace '------' with the path such that "DM890_assignment_1" is your working directory
%cd '/content/gdrive/My Drive/AML_Exam/Problem2'

/content/gdrive/.shortcut-targets-by-id/1osKmyrDLORFSpes1mwrU-W7OXM-to6bK/AML_Exam/Problem2


# Function to sample the data, and data paths

In [ ]:
def dataset_name(file_path):
  dataset_name = os.path.splitext(os.path.basename(file_path))[0].replace('_5.json', '')
  return dataset_name

In [ ]:
def loading_gz(file_path, printing = False, load_all = False, max_load_samples = np.inf, max_samples_per_cat = 2000):
  my_keys = ['overall', 'reviewText', 'summary']
  with gzip.open(file_path, 'rt', encoding='utf-8') as g:
          # Initialize an empty list to store sampled rows
          sampled_data = []
          for i, line in enumerate(g):
              if printing:
                if i % 1000 == 0:
                    print(i)
              data = json.loads(line)
              sampled_data.append({k: data[k] for k in my_keys & data.keys()})
              if i > max_load_samples:
                break
          sampled_data = pd.DataFrame(sampled_data)
          sampled_data.dropna(inplace = True)

          if load_all:
            return sampled_data

          nMax = sampled_data.groupby('overall').count().min()[0] * 3
          #my_max = min(nMax, 2000)
          res = sampled_data.groupby('overall').apply(lambda x: x.sample(n = min(nMax, len(x), max_samples_per_cat))).reset_index(drop = True)

          return res


In [ ]:
# List of file paths for your 29 datasets
file_paths_train = [
    '/content/gdrive/My Drive/AML_Exam/Problem2/data/Arts_Crafts_and_Sewing_5.json.gz',
    '/content/gdrive/My Drive/AML_Exam/Problem2/data/Prime_Pantry_5.json.gz'
]

path_val  = '/content/gdrive/My Drive/AML_Exam/Problem2/data/All_Beauty_5.json.gz'

path_test = '/content/gdrive/My Drive/AML_Exam/Problem2/data/Luxury_Beauty_5.json.gz'

data_path_dict = {dataset_name(k):k for k in file_paths_train}


# Loading data

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Defining Encoding function

In [ ]:
def my_encoder(x_train, x_test = None, MAX_VOCAB_SIZE = 1000, OUTPUT_LENGTH = None):
  encoder = tf.keras.layers.TextVectorization(max_tokens=MAX_VOCAB_SIZE, output_sequence_length = OUTPUT_LENGTH)
  encoder.adapt(x_train)
  # vocab = np.array(encoder.get_vocabulary())

  encoded_x = encoder(x_train)

  if not pd.isnull(x_test).all():
    encoded_x_test = encoder(x_test)
    return encoded_x, encoded_x_test
  else:
    return encoded_x

# WANDB initialization

In [ ]:
#wandb.login()
os.environ["WANDB_API_KEY"] = "f1dd10d2eaa5cc353bbe282583f7fc58b9b6c733"
sweep_navn = "vocab and length"

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': sweep_navn,
    'method': 'grid', #grid, random
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        'training_set_name': {'values': list(data_path_dict.keys())},
        'vocab_size': {'values': [500,1000,1500,2000]},
        'review_length': {'values': [None, 50, 100, 150, 200]},
        #'epochs': {'values': [20]},
        #'batch_size': {'values': [64]},
        #'dropout': {'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]},
        #'learning_rate': {'values': [0.0075]},
        #'optimizer': {'values': ['adam', 'SGD']},
        #'activation': {'values': ['relu']},
        #'hidden_layer_size': {'values': [512, 1024]},
        #'hidden_layers': {'min': 2, 'max': 5}
    }
}

sweep_id = wandb.sweep(sweep_config, project="RNN_preliminary_tests", entity='coffeegang')

Create sweep with ID: rd1v16t6
Sweep URL: https://wandb.ai/coffeegang/RNN_preliminary_tests/sweeps/rd1v16t6


## Setting up RNN with gpu

In [ ]:
def train_model():
  config_defaults = {
    'training_set_name': '',
    'vocab_size': 1000,
    'review_length': None,
    #'train_size': int(50000/5),
    'epochs': 50,
    'batch_size': 64,
    'learning_rate': 0.0075,
    'optimizer': 'adam',
    'validation_split': 0.3,
  }

  with tf.device('/gpu:0'):
    # Initialize a new wandb run
    wandb.init(config=config_defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=config.vocab_size, output_dim=128),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        #tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(5, activation='softmax'),
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate = config.learning_rate)

    model.compile(
        loss = "sparse_categorical_crossentropy",
        optimizer = optimizer,
        metrics = ['accuracy'],
    )

    load_train = loading_gz(data_path_dict[config.training_set_name], max_load_samples = 1000000, max_samples_per_cat=config.train_size)
    x_train = load_train['reviewText']
    y_train = load_train['overall'] - 1 #the minus 1 is to adjust for the output of the model being [0,4]

    load_val = loading_gz(path_val, load_all=True)
    x_val = load_val['reviewText']
    y_val = load_val['overall'] - 1 #the minus 1 is to adjust for the output of the model being [0,4]

    encoded_train, encoded_val  = my_encoder(x_train, x_val, MAX_VOCAB_SIZE = config.vocab_size, OUTPUT_LENGTH = config.review_length)

    wandb.log({"size-train": len(x_train), "size-val": len(x_val)})

    model.fit(encoded_train, y_train.values,
              epochs = config.epochs,
              batch_size = config.batch_size,
              validation_data=(encoded_val, y_val.values),
              callbacks=[WandbCallback(save_model = False),
                          tf.keras.callbacks.EarlyStopping(
                              monitor="accuracy",
                              #min_delta=0.01,
                              patience=3,
                              #mode="max"
                          )])

In [ ]:
wandb.agent(sweep_id, train_model)

wandb: Agent Starting Run: 0ebx07s5 with config:
wandb: 	review_length: None
wandb: 	training_set_name: Arts_Crafts_and_Sewing
wandb: 	vocab_size: 500
wandb: Currently logged in as: aksel (coffeegang). Use `wandb login --relogin` to force relogin


Epoch 1/50
782/782 [==============================] - 103s 128ms/step - loss: 1.2365 - accuracy: 0.4544 - val_loss: 1.0090 - val_accuracy: 0.6081
Epoch 2/50
782/782 [==============================] - 90s 115ms/step - loss: 1.1327 - accuracy: 0.5001 - val_loss: 0.8740 - val_accuracy: 0.6853
Epoch 3/50
782/782 [==============================] - 90s 115ms/step - loss: 1.0916 - accuracy: 0.5179 - val_loss: 1.0390 - val_accuracy: 0.6366
Epoch 4/50
782/782 [==============================] - 90s 115ms/step - loss: 1.0664 - accuracy: 0.5367 - val_loss: 0.8898 - val_accuracy: 0.7133
Epoch 5/50
782/782 [==============================] - 90s 115ms/step - loss: 1.0501 - accuracy: 0.5444 - val_loss: 1.1043 - val_accuracy: 0.5535
Epoch 6/50
782/782 [==============================] - 90s 114ms/step - loss: 1.0371 - accuracy: 0.5521 - val_loss: 1.1153 - val_accuracy: 0.6110
Epoch 7/50
782/782 [==============================] - 90s 115ms/step - loss: 1.0248 - accuracy: 0.5576 - val_loss: 1.0347 - val_a

accuracy,▁▄▅▆▆▇▇▇████████
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
loss,█▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁
size-train,▁
size-val,▁
val_accuracy,▃▇▅█▁▄▅▄▆▇▅▄▄▂▅▄
val_loss,▄▁▄▁▆▆▄▆▃▂▅▅▅█▄▇
accuracy,0.56812
best_epoch,1
best_val_loss,0.87396
epoch,15


wandb: Agent Starting Run: 8nbkykcx with config:
wandb: 	review_length: None
wandb: 	training_set_name: Arts_Crafts_and_Sewing
wandb: 	vocab_size: 1000


Epoch 1/50
782/782 [==============================] - 124s 154ms/step - loss: 1.2084 - accuracy: 0.4647 - val_loss: 0.9355 - val_accuracy: 0.7153
Epoch 2/50
782/782 [==============================] - 107s 136ms/step - loss: 1.0920 - accuracy: 0.5158 - val_loss: 1.1019 - val_accuracy: 0.5967
Epoch 3/50
782/782 [==============================] - 106s 135ms/step - loss: 1.0522 - accuracy: 0.5379 - val_loss: 0.8550 - val_accuracy: 0.6990
Epoch 4/50
782/782 [==============================] - 105s 135ms/step - loss: 1.0206 - accuracy: 0.5541 - val_loss: 0.9296 - val_accuracy: 0.6779
Epoch 5/50
782/782 [==============================] - 105s 134ms/step - loss: 0.9988 - accuracy: 0.5660 - val_loss: 0.9617 - val_accuracy: 0.6705
Epoch 6/50
782/782 [==============================] - 105s 134ms/step - loss: 0.9786 - accuracy: 0.5755 - val_loss: 1.0563 - val_accuracy: 0.6471
Epoch 7/50
782/782 [==============================] - 105s 134ms/step - loss: 0.9630 - accuracy: 0.5826 - val_loss: 0.9526 -

...

accuracy,▁▃▃▄▄▅▅▆▆▆▇▇▇▇▇██████████████
epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
loss,█▆▆▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
size-train,▁
size-val,▁
val_accuracy,█▆█▂▅▅▆▄▅▂▅▂▂▂▃▄▂▃▄▁▂▃▃▂▂▃▃▃▃
val_loss,▁▁▁▄▂▄▂▃▃▅▃▆▅▅▆▅▆▆▆█▆▆▆▇▇▆▇▇▇
accuracy,0.73106
best_epoch,2
best_val_loss,0.72125
epoch,28


wandb: Agent Starting Run: andkuxs3 with config:
wandb: 	review_length: 200
wandb: 	training_set_name: Prime_Pantry
wandb: 	vocab_size: 2000


Epoch 1/50
556/556 [==============================] - 33s 53ms/step - loss: 1.1665 - accuracy: 0.4898 - val_loss: 0.9088 - val_accuracy: 0.6887
Epoch 2/50
556/556 [==============================] - 13s 23ms/step - loss: 1.0341 - accuracy: 0.5499 - val_loss: 0.8397 - val_accuracy: 0.6899
Epoch 3/50
556/556 [==============================] - 12s 21ms/step - loss: 0.9741 - accuracy: 0.5804 - val_loss: 1.0296 - val_accuracy: 0.6106
Epoch 4/50
556/556 [==============================] - 11s 19ms/step - loss: 0.9212 - accuracy: 0.6049 - val_loss: 0.9095 - val_accuracy: 0.6524
Epoch 5/50
556/556 [==============================] - 10s 18ms/step - loss: 0.8761 - accuracy: 0.6237 - val_loss: 1.0190 - val_accuracy: 0.6222
Epoch 6/50
556/556 [==============================] - 10s 18ms/step - loss: 0.8497 - accuracy: 0.6398 - val_loss: 0.8651 - val_accuracy: 0.6705
Epoch 7/50
556/556 [==============================] - 10s 18ms/step - loss: 0.8026 - accuracy: 0.6636 - val_loss: 1.0216 - val_accuracy:

accuracy,▁▃▃▄▅▅▆▆▆▆▇▇▇▇▇██████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▆▆▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
size-train,▁
size-val,▁
val_accuracy,██▃▅▃▇▃▅▅▃▂▁▂▅▃▂▅▃▂▂▃▃▁▂▄
val_loss,▂▁▄▂▄▁▄▃▄▄▅▇▆▅▆▇▆▇█▇▇███▇
accuracy,0.73852
best_epoch,1
best_val_loss,0.8397
epoch,24


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
